### Social Network Recommendations: People You May Know (PYMK)

In this example, we'll build a predictive capability for recommending potential connections in a social network using Amazon Neptune and Gremlin queries. This technique leverages *triadic closure*, a phenomenon where friends of friends are likely to become friends themselves over time.

**Triadic Closure in Social Networks:**
Consider a social network with members Bill, Terry, and Sarah. Terry is friends with both Bill and Sarah, creating a triangle. The connection among them suggests that Sarah and Bill may already know each other or will likely connect soon. Bill serves as a mutual friend, providing both the *means* (opportunity) and *motive* (trust) for Sarah and Bill to become friends.

**Implementing PYMK:**
When a user logs into the system, we can analyze their vertex in the graph and traverse their friend-of-a-friend network. By identifying potential connections not currently in their network, we can recommend friends based on shared relationships. The more connections extending from a user through their friends to others, the higher the likelihood of forming new connections.

This approach not only enhances user experience but also strengthens community ties within the network.

### Installation

First, we'll reference a couple of helper libraries, _neptune.py_ and _visualisation.py_. Then we'll clear any existing data from our Neptune cluster, and create a refernce to `g`, a graph traversal source, which we then use in subsequent queries:

In [ ]:
%run '../util/neptune.py'
%run '../util/visualisation.py'

neptune.clear()
g = neptune.graphTraversal()

How do we know which Neptune cluster to access? The methods above use a couple of environment variables, NEPTUNE_CLUSTER_ENDPOINT and NEPTUNE_CLUSTER_PORT, which were initialised when we first created this notebook instance.

If you want to refer to a different Neptune cluster, you can supply the endpoint and port to the `clear()` and `graphTraversal()` methods:

```
neptune.clear(neptune_endpoint=<NEPTUNE_ENDPOINT>, neptune_port=<NEPTUNE_PORT>)
g = neptune.graphTraversal(neptune_endpoint=<NEPTUNE_ENDPOINT>, neptune_port=<NEPTUNE_PORT>)
```

### Create a Social Network

Next, we'll create a small social network. Note that the script below comprises a single statement. All the vertices and edges here will be created in the context of a single transaction.

In [ ]:
// Add Users with Indian Names
g.
addV('User').property('name','Anmol').property('birthdate', '1998-01-01'). \
addV('User').property('name','Suresh').property('birthdate', '1992-05-03'). \
addV('User').property('name','Rajesh').property('birthdate', '1989-10-21'). \
addV('User').property('name','Priya').property('birthdate', '1998-01-17'). \
addV('User').property('name','Rahul').property('birthdate', '2001-08-14'). \
addV('User').property('name','Neha').property('birthdate', '1998-03-05'). \
addV('User').property('name','Amit').property('birthdate', '2002-12-04'). \
addV('User').property('name','Kavita').property('birthdate', '1995-02-12'). \
addV('User').property('name','Arjun').property('birthdate', '2001-02-27'). \
addV('User').property('name','Tara').property('birthdate', '1989-10-02'). \
addV('User').property('name','Vikram').property('birthdate', '1992-06-30'). \
addV('User').property('name','Aisha').property('birthdate', '2000-05-13'). \
addV('User').property('name','Sanya').property('birthdate', '1997-01-27'). \
addV('User').property('name','Sameer').property('birthdate', '1989-11-02'). \
addV('User').property('name','Poonam').property('birthdate', '1994-03-08'). \
addV('User').property('name','Aditya').property('birthdate', '2002-07-23'). \
addV('User').property('name','Nisha').property('birthdate', '1988-11-04'). \
addV('User').property('name','Harsh').property('birthdate', '1996-03-15'). \
addV('User').property('name','Simran').property('birthdate', '2003-08-21'). \

// Add Friend Relationships
V().hasLabel('User').has('name','Suresh').as_('a').V().hasLabel('User').has('name','Anmol').addE('FRIEND').to('a').property('strength',1). \
V().hasLabel('User').has('name','Rahul').as_('a').V().hasLabel('User').has('name','Anmol').addE('FRIEND').to('a').property('strength',2). \
V().hasLabel('User').has('name','Tara').as_('a').V().hasLabel('User').has('name','Anmol').addE('FRIEND').to('a').property('strength',3). \
V().hasLabel('User').has('name','Arjun').as_('a').V().hasLabel('User').has('name','Rahul').addE('FRIEND').to('a').property('strength',1). \
V().hasLabel('User').has('name','Kavita').as_('a').V().hasLabel('User').has('name','Rajesh').addE('FRIEND').to('a').property('strength',2). \
V().hasLabel('User').has('name','Kavita').as_('a').V().hasLabel('User').has('name','Priya').addE('FRIEND').to('a').property('strength',3). \
V().hasLabel('User').has('name','Aisha').as_('a').V().hasLabel('User').has('name','Priya').addE('FRIEND').to('a').property('strength',1). \
V().hasLabel('User').has('name','Vikram').as_('a').V().hasLabel('User').has('name','Kavita').addE('FRIEND').to('a').property('strength',2). \
V().hasLabel('User').has('name','Sanya').as_('a').V().hasLabel('User').has('name','Rahul').addE('FRIEND').to('a').property('strength',3). \
V().hasLabel('User').has('name','Harsh').as_('a').V().hasLabel('User').has('name','Vikram').addE('FRIEND').to('a').property('strength',3). \
V().hasLabel('User').has('name','Vikram').as_('a').V().hasLabel('User').has('name','Suresh').addE('FRIEND').to('a').property('strength',1). \
V().hasLabel('User').has('name','Simran').as_('a').V().hasLabel('User').has('name','Aisha').addE('FRIEND').to('a').property('strength',2). \
V().hasLabel('User').has('name','Simran').as_('a').V().hasLabel('User').has('name','Sameer').addE('FRIEND').to('a').property('strength',3). \
V().hasLabel('User').has('name','Anmol').as_('a').V().hasLabel('User').has('name','Sameer').addE('FRIEND').to('a').property('strength',1). \
V().hasLabel('User').has('name','Harsh').as_('a').V().hasLabel('User').has('name','Sameer').addE('FRIEND').to('a').property('strength',2). \
V().hasLabel('User').has('name','Harsh').as_('a').V().hasLabel('User').has('name','Sanya').addE('FRIEND').to('a').property('strength',3). \
V().hasLabel('User').has('name','Sanya').as_('a').V().hasLabel('User').has('name','Aisha').addE('FRIEND').to('a').property('strength',1). \
V().hasLabel('User').has('name','Nisha').as_('a').V().hasLabel('User').has('name','Harsh').addE('FRIEND').to('a').property('strength',2). \
V().hasLabel('User').has('name','Nisha').as_('a').V().hasLabel('User').has('name','Sanya').addE('FRIEND').to('a').property('strength',3). \
next()


This is what the network looks like:
<img src="SocialNetwork.webp">


### Create a Recommendation

Let's now create a PYMK recommendation for a specific user.

In the query below, we're finding the vertex that represents our user. We're then traversing `FRIEND` relationships (we don't care about relationship direction, so we're using `both()`) to find that user's immediate friends. We're then traversing another hop into the graph, looking for friends of those friends who _are not currently connected to our user_ (i.e., we're looking for the unclosed triangles).

We then count the paths to these candidate friends, and order the results based on the number of times we can reach a candidate via one of the user's immediate friends.

In [ ]:
user = 'Terry'

recommendations = (g.V().hasLabel('User').has('name', user).as_('user').  
  both('FRIEND').aggregate('friends').  
  both('FRIEND').
    where(P.neq('user')).where(P.without('friends')).  
  groupCount().by('name').  
  order(Scope.local).by(Column.values, Order.decr).
  next())

for key in recommendations.keys():
    print(key + ': ' + str(recommendations[key]))

### Visualise the Results

Using _matplotlib_ we can visualise the results of a query.

The sample code below uses the same traversal patterns as before to identify PYMK candidates, but instead of creating a resultset based on the number of times we reach a candidate, we create two projections: a list of candidate friends, and a list of paths to those candidate friends. We then use these lists to show the local network surrounding our user, with the PYML candidates highlighted in green.

_visualisation.py_ contains the visualisation code. You can use this code as-is, or adapt it to your own needs.

In [ ]:
results = g.V().hasLabel('User').has('name', user).as_('user'). \
  both('FRIEND').aggregate('friends'). \
  both('FRIEND'). \
    where(P.neq('user')).where(P.without('friends')).aggregate('not-friends').by('name'). \
  path().by('name').aggregate('p'). \
  project('not-friends', 'paths'). \
  by(__.select('not-friends').unfold().dedup().fold()). \
  by(__.select('p')).next()

notFriends = results['not-friends']
paths = results['paths']

def formatter(label, colors):
    if label in notFriends:
        colors.append('#11cc77')
    else:
        colors.append('yellow')

visualisation.plotPaths(paths, formatter)

### Using Friendship Strength to Improve Recommendations

What if we wanted to base our recommendations only on resonably strong friendship bonds?

If you look at the Gremlin we used to create our graph, you'll see that each `FRIEND` edge has a `strength` property. In the following query, the traversal applies a predicate to this `strength` property. Note that we use `bothE()` rather than `both()` to position the traversal on an edge, where we then apply the predicate. We proceed only where `strength` is greater than one.

In [ ]:
recommendations = (g.V().hasLabel('User').has('name', user).as_('user').  
  bothE('FRIEND').    
    has('strength', P.gt(1)).otherV().    
    aggregate('friends').  
  bothE('FRIEND').    
    has('strength', P.gt(1)).otherV().    
    where(P.neq('user')).where(P.without('friends')).  
  groupCount().by('name').  
  order(Scope.local).by(Column.values, Order.decr).
  next())

for key in recommendations.keys():
    print(key + ': ' + str(recommendations[key]))

Because we discount weak friendships even when traversing to immediate friends, this query can sometimes end up recommending people that have a weak direct tie to our user. But that makes sense in the context of our social domain: one of our close friebnds has a strong friendship with one of the people with whom we have a weak connection; therefore, we might predict that over time this weak bond will grow stronger.

### Using Subgraph Strategies to Filter Edges

The code above required us to change the initial query in order to apply a predicate to every edge. As an alternative, we can use a [SubgraphStrategy](http://tinkerpop.apache.org/docs/current/reference/#_subgraphstrategy) to supply a filter that will be applied to every edge we traverse. Using the SubgraphStrategy, we can reuse our original query instead of having to rewrite it with `bothE()`.

First, we create a new traversal source with a SubgraphStrategy applied:

In [ ]:
from gremlin_python.process.strategies import *

g2 = g.withStrategies(SubgraphStrategy(edges=__.has('strength', P.gt(1))))

Then we rerun the original query, but using the new traversal source (`g2` rather than `g`):

In [ ]:
recommendations = (g2.V().hasLabel('User').has('name', user).as_('user').  
  both('FRIEND').aggregate('friends').  
  both('FRIEND').
    where(P.neq('user')).where(P.without('friends')).  
  groupCount().by('name').  
  order(Scope.local).by(Column.values, Order.decr).
  next())

for key in recommendations.keys():
    print(key + ': ' + str(recommendations[key]))